In [1]:
import os
import json
import numpy as np
import pandas as pd
import scipy.sparse as spr

목차

- 1. 데이터 처리 (불러오기,추가,수정,변환)
- 2. 플레이리스트로 추천 (협업필터링)

# 데이터 처리

## train,song,genre data 불러오기

In [2]:
with open('data/train.json',encoding='utf-8-sig') as f:
    train_dict = json.load(f)
    
with open('data/song_meta.json',encoding='utf-8-sig') as f:
    song_dict = json.load(f)
    
with open('data/genre_gn_all.json',encoding='utf-8-sig') as f:
    genre_dict = json.load(f)
    
# dict -> dataframe 변환
train_df = pd.DataFrame.from_dict(train_dict)
song_df = pd.DataFrame.from_dict(song_dict)

## train_df 전처리

In [3]:
train_df.head(3)

,tags,id,plylst_title,songs,like_cnt,updt_date
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000
1,"[추억, 회상]",10532,요즘 너 말야,"[432406, 675945, 497066, 120377, 389529, 24427...",1,2014-12-02 16:19:42.000
2,"[까페, 잔잔한]",76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[83116, 276692, 166267, 186301, 354465, 256598...",17,2017-08-28 07:09:34.000


### train_df 플레이리스트에 포함된 태그의 갯수, 곡수 컬럼 추가

In [4]:
# 플레이리스트 곡수 컬럼 추가
train_df['tags_cnt'] = train_df['tags'].map(lambda x : len(x))

# 플레이리스트 태그수 컬럼 추가
train_df['songs_cnt'] = train_df['songs'].map(lambda x : len(x))

### train_df 에 포함된 곡들 (중복포함,중복제거)

In [5]:
from itertools import chain

# 플레이리스트 포함된 노래 중복포함
songs_duplicate = chain.from_iterable(train_df['songs'].tolist())

# 플레이리스트 포함된 노래 중복제거
songs_unique = list(set(songs_duplicate))

### train_df 에 포함된 태그들 (중복포함,중복제거)

In [6]:
# 플레이리스트 포함된 태그 중복포함
tags_duplicate = list(chain.from_iterable(train_df['tags'].tolist()))

# 플레이리스트 포함된 태그 중복제거
tags_unique = list(set(tags_duplicate))

### tag에 새로운 id부여, new_tags_id 컬럼 생성

In [7]:
# { 태그 : 새로운id } 딕셔너리
tag_to_id = dict(zip(tags_unique,range(0,len(tags_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_tag = dict(zip(range(0,len(tags_unique)),tags_unique))

train_df['new_tags_id'] = train_df['tags'].map(lambda x : [tag_to_id[v] for v in x])

### songs에 새로운 id부여, new_songs_id 컬럼 생성

In [8]:
# { 노래 : 새로운id } 딕셔너리
song_to_id = dict(zip(songs_unique,range(0,len(songs_unique))))

# { 새로운id : 태그 } 딕셔너리
id_to_song = dict(zip(range(0,len(songs_unique)),songs_unique))

train_df['new_songs_id'] = train_df['songs'].map(lambda x : [song_to_id[v] for v in x])

In [9]:
train_df.head(1)

,tags,id,plylst_title,songs,like_cnt,updt_date,tags_cnt,songs_cnt,new_tags_id,new_songs_id
0,[락],61281,여행같은 음악,"[525514, 129701, 383374, 562083, 297861, 13954...",71,2013-12-19 18:36:19.000,1,19,[3056],"[456704, 112732, 333158, 488440, 258853, 12127..."


In [10]:
train_df = train_df[['id','plylst_title','tags','new_tags_id','songs','new_songs_id','tags_cnt','songs_cnt','like_cnt','updt_date']]
train_df.head(3)

,id,plylst_title,tags,new_tags_id,songs,new_songs_id,tags_cnt,songs_cnt,like_cnt,updt_date
0,61281,여행같은 음악,[락],[3056],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[11002, 26830]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[12076, 11561]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


In [11]:
train_df.columns = ['플리_id','플리제목','태그','새태그_id','노래_id','새노래_id','태그수','노래수','좋아요수','갱신일']
train_df.head(3)

,플리_id,플리제목,태그,새태그_id,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일
0,61281,여행같은 음악,[락],[3056],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000
1,10532,요즘 너 말야,"[추억, 회상]","[11002, 26830]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[12076, 11561]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000


### 변수 정리

<현재>
 
- tag_to_id : { 태그 : 새로운id } 딕셔너리
- id_to_tag : { 새로운id : 태그 } 딕셔너리
- song_to_id : { 노래 : 새로운id } 딕셔너리
- id_to_song : { 새로운id : 노래 } 딕셔너리

<이후>
- code_to_genre {장르코드: 장르} 딕셔너리
- genre_to_code {장르 : 장르코드} 딕셔너리
- gcode_to_id  {장르코드 : 장르_id} 딕셔너리
- id_to_gcode  {장르_id : 장르코드} 딕셔너리

# 장르와 코드로 협업필터링 추천해보자

## genre 앞의 네글자로 대장르만 가져오기

In [12]:
genre_big = {}

# 모든 장르 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨 뒤 두자리가 00이면 대장류로 분류
    if k[-2:] == '00':
        
        # 맨앞 네자리를 키로 하는 대장류 딕셔너리 값 추가
        genre_big[k[:4]] = v
#genre_big

## 세부장르에 대분류 장르의 이름 넣기

In [13]:
code_to_genre = {}

# 모든 딕셔너리를 돌면서
for k,v in genre_dict.items():
    
    # 맨뒤 두자리가 00이 아니면 대장류가 아닌거임!
    if k[-2:] != '00':
        
        # 그럴떈 아까만든 대장르 딕셔너리의 대장류 이름을 추가해서 이름을 수정해서 다시 넣어줌
        new_value = genre_big[k[:4]]+'_'+v
        code_to_genre[k] = new_value
        
    # 대장류면 그대로
    else:
        code_to_genre[k] = v

#code_to_genre

## 장르 딕셔너리 키 밸류 바꾸기 -> 나중에 장르 이름으로 코드 찾을 거기 때문임!

In [14]:
genre_to_code = {v:k for k,v in code_to_genre.items()}

#genre_to_code 

## Train과 장르 합치기
- 플레이리스트 안에 들어있는 노래의 장르를 중복제거 해서 컬럼에 추가
- 35초 정도 걸렸음

In [15]:
genre_basket = song_df['song_gn_dtl_gnr_basket'].to_numpy()
song_id_temp = train_df['노래_id'].to_numpy() 

In [16]:
train_df['장르'] = pd.Series([list(set(genre_basket[id])) for id_list in song_id_temp for id in id_list])

In [17]:
train_df.head(3)

,플리_id,플리제목,태그,새태그_id,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르
0,61281,여행같은 음악,[락],[3056],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN1402, GN1401]"
1,10532,요즘 너 말야,"[추억, 회상]","[11002, 26830]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN0902, GN0901, GN1001]"
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[12076, 11561]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN1001, GN1012, GN1005]"


## 새로운 장르 코드 생성

In [18]:
genre_code_unique = list(genre_dict.keys())
genre_code_to_id = dict(zip(genre_code_unique,range(len(genre_code_unique))))
id_to_genre_code = dict(zip(range(len(genre_code_unique)),genre_code_unique))

In [19]:
train_df['장르_id'] = train_df['장르'].map(lambda x : [ genre_code_to_id[i] for i in x] )

In [20]:
train_df.head(3)

,플리_id,플리제목,태그,새태그_id,노래_id,새노래_id,태그수,노래수,좋아요수,갱신일,장르,장르_id
0,61281,여행같은 음악,[락],[3056],"[525514, 129701, 383374, 562083, 297861, 13954...","[456704, 112732, 333158, 488440, 258853, 12127...",1,19,71,2013-12-19 18:36:19.000,"[GN1402, GN1401]","[111, 110]"
1,10532,요즘 너 말야,"[추억, 회상]","[11002, 26830]","[432406, 675945, 497066, 120377, 389529, 24427...","[375894, 587314, 431997, 104605, 338568, 21226...",2,42,1,2014-12-02 16:19:42.000,"[GN0902, GN0901, GN1001]","[58, 57, 66]"
2,76951,"편하게, 잔잔하게 들을 수 있는 곡.-","[까페, 잔잔한]","[12076, 11561]","[83116, 276692, 166267, 186301, 354465, 256598...","[72132, 240434, 144495, 161861, 307991, 222934...",2,28,17,2017-08-28 07:09:34.000,"[GN1001, GN1012, GN1005]","[66, 77, 70]"


## 태그_id 하나당 어떤 장르가 포함되었는지 araboja

- 각 태그가 몇번 출몰 했는지 알아야 함

### 태그 출몰 횟수 구하기

In [21]:
tag_id_duplicate = list(chain.from_iterable(train_df['새태그_id'].tolist()))
tag_id_unique = list(id_to_tag.keys())

In [22]:
# 각 태그가 몇번 출몰 했는지 세어 놓은 딕셔너리

from collections import Counter

tag_count = dict(Counter(tag_id_duplicate))

tag_count = {i:tag_count[i] for i in range(len(tag_count))}

In [23]:
# 태그_id에 달린 장르id들을 넣을 딕셔너리 초기화
tag_id_to_genre_ids = {i:[] for i in range(len(tag_id_unique))}

In [24]:
# 태그_id에 달린 장르id들을 list로 넣어주기
for tag_ids , genre_ids in zip(train_df['새태그_id'].tolist(),train_df['장르_id'].tolist()):
    for tag_id in tag_ids:
        tag_id_to_genre_ids[tag_id].extend(genre_ids)

In [25]:
# 장르id가 중복된 횟수 순서로 정렬

for i in range(len(tag_id_to_genre_ids)):
    genre_count = Counter(tag_id_to_genre_ids[i])
    genre_count_sorted = sorted(genre_count.items(),key=(lambda x:x[1]),reverse=True)

    temp_list = []
    
    if len(genre_count_sorted) > 9:
        for k,v in genre_count_sorted[:10]:
            temp_list.append((k,v))
    else:
        for k,v in genre_count_sorted:
            temp_list.append((k,v))
            
    tag_id_to_genre_ids[i] = temp_list

{0: [(156, 1), (162, 1)],
 1: [(141, 1), (134, 1), (129, 1)],
 2: [(102, 1), (101, 1), (104, 1)],
 3: [(5, 1), (1, 1)],
 4: [(91, 1),
  (92, 1),
  (81, 1),
  (82, 1),
  (121, 1),
  (230, 1),
  (118, 1),
  (226, 1),
  (1, 1),
  (5, 1)],
 5: [(156, 1), (81, 1)],
 6: [(1, 3), (5, 2), (175, 1), (165, 1), (164, 1), (4, 1), (121, 1), (118, 1)],
 7: [(91, 1)],
 8: [(3, 1),
  (1, 1),
  (102, 1),
  (101, 1),
  (146, 1),
  (147, 1),
  (145, 1),
  (15, 1),
  (13, 1)],
 9: [(81, 2), (84, 1), (83, 1), (241, 1), (82, 1)],
 10: [(19, 1), (21, 1)],
 11: [(136, 1),
  (129, 1),
  (230, 1),
  (226, 1),
  (1, 1),
  (5, 1),
  (228, 1),
  (84, 1),
  (83, 1),
  (81, 1)],
 12: [(5, 1), (1, 1)],
 13: [(1, 4),
  (4, 3),
  (5, 2),
  (118, 2),
  (66, 1),
  (227, 1),
  (226, 1),
  (7, 1),
  (231, 1),
  (11, 1)],
 14: [(33, 1),
  (38, 1),
  (15, 1),
  (13, 1),
  (5, 1),
  (1, 1),
  (226, 1),
  (7, 1),
  (231, 1),
  (228, 1)],
 15: [(119, 1), (118, 1), (4, 1)],
 16: [(5, 1), (1, 1)],
 17: [(38, 1), (31, 1), (33, 1),

In [26]:
# 포함된 장르의 개수를 가지고 있는 list
genre_len = [len(v) for v in tag_id_to_genre_ids.values()]

In [27]:
#  태그별 장르포함 횟수만큼 반복  * 2번 태그는 7번 나왔으니 2를 7번 찍기
row = np.repeat(range(len(tag_id_to_genre_ids)),genre_len)
len(row)

137062

In [28]:
# 장르를 하나씩 뽑아서 넣어주기
# map ,filter 써서 코드 개선시키기 파이썬 내장함수
col =  [id[0] for ids in tag_id_to_genre_ids.values() for id in ids]
len(col)

137062

In [29]:
# 태그 마다 달린 장르의 총길이 만큼 1찍기
dat = np.repeat(1, sum(genre_len))
len(dat)

137062

In [30]:
train_tags_A = spr.csr_matrix((dat, (row, col)), shape=(train_df['태그수'].sum(), len(genre_code_to_id)))

train_tags_A_T = train_tags_A.T.tocsr()

# 내가 고른 장르와 연관된 장르를 뽑자@!@

## 나의 장르 리스트 만들기

In [31]:
def recom_genres(tag):

    my_genre_ids = tag_id_to_genre_ids[tag_to_id[tag]]
    
    my_genre_ids
    
    # 내가 뽑은 장르 id 만 1인 1차원 희소행렬

    p = np.zeros((len(genre_code_to_id),1))

    for id,count in my_genre_ids:
        p[id]=1
        
    # 희소행렬과 dot 연산을 통해 각 태그와 겹치는 장르들의 갯수를 추출
    val = train_tags_A.dot(p).reshape(-1)
    
    cand_gen = train_tags_A_T.dot(val)
    
    # 1차원으로 바꿔서 중복된 장르를 가진 태그 상위 150개 도출
    cand_gen_idx = cand_gen.reshape(-1).argsort()[-150:][::-1]

    # 비슷한 장르 10개 도출
    cand_gen_idx = cand_gen_idx[:10]

    cand_gen_idx
    
    # 원래 노래 id값으로 복원
    result_ids = [id_to_genre_code[i] for i in cand_gen_idx]
    
    result_genres = [(code_to_genre[i]) for i in result_ids]
    
    return result_genres

In [32]:
recom_genres('기분좋은')

['발라드_세부장르전체',
 "발라드_'10-",
 '아이돌_세부장르전체',
 '인디음악_세부장르전체',
 '댄스_세부장르전체',
 "인디음악_'10-",
 'POP_세부장르전체',
 '랩/힙합_세부장르전체',
 'OST_세부장르전체',
 '아이돌_댄스']